<table>
    <tr>
        <td><img src="./imagenes/Macc.png" width="400"/></td>
        <td>&nbsp;</td>
        <td>
            <h1 style="color:blue;text-align:left">Inteligencia Artificial</h1></td>
        <td>
            <table><tr>
            <tp><p style="font-size:150%;text-align:center">Notebook</p></tp>
            <tp><p style="font-size:150%;text-align:center">Bases de conocimiento</p></tp>
            </tr></table>
        </td>
    </tr>
</table>

---


## Objetivo <a class="anchor" id="inicio"></a>

En este notebook veremos una manera de implementar una base de conocimiento sobre la cual se ejecutan los algoritmos de deducción forward-chaining y backward-chaining. 

## Secciones

Desarrollaremos la explicación en las siguientes secciones:

* [Base de conocimiento.](#base)
* [Forward chaining.](#forward)
* [Backward chaining.](#backward)
* [TELL y ASK.](#tell)


## Base de conocimiento <a class="anchor" id="base"></a>

([Volver al inicio](#inicio))

Una base de conocimiento es una estructura sobre la cual se pueden hacer consultas de objetivos. Más adelante usaremos las estrategias de forward y backward chaining para hacer las consultas. Lo que necesitamos ahora es una implementación de la base de conocimiento en python. 

Una base de conocimiento tiene dos partes: 

* Los hechos, que son literales que ya sabemos como verdaderos; 
* Las reglas, que representan el conocimiento que se aplicará sobre los datos para obtener nuevos hechos.

Implementaremos una base de conocimiento mediante la clase `LPQuery`, que se encuentra en la librería `logica`:

In [3]:
from logica import LPQuery

Inicializamos una base de conocimiento con el ejemplo de las diapositivas:

In [4]:
formulas = ['q>r','pYs>r','s','p']
base_ejemplo = LPQuery(formulas)
print(base_ejemplo)

Hechos:
s
p

Reglas:
q>r
pYs>r



La clase `LPQuery` tiene los siguientes atributos:

* hechos: lista de literales que ya sabemos como verdaderos.
* reglas: lista de reglas que representan el conocimiento.
* atomos: lista de los átomos que aparecen en los datos y las reglas.

También tiene los siguientes métodos:

* reglas_aplicables(`literal`): que devuelve una lista de reglas en la base cuya cabeza es el `literal`.
* test_objetivo(`literal`): que devuelve verdadero si el `literal` está en los datos. 
* TELL(`formula`): que incluye en la base de conocimiento la `formula`en el lugar adecuado, dependiendo de si es un dato o una regla.

In [5]:
reglas = base_ejemplo.reglas
print(reglas[1].nombre)
print(reglas[1].antecedente)
print(reglas[1].consecuente)

pYs>r
['p', 's']
r


In [6]:
base_ejemplo.atomos

['q', 'r', 'p', 's']

## Forward chaining <a class="anchor" id="forward"></a>

([Volver al inicio](#inicio))

El siguiente es el pseudo código de la función `pl_fc_entails`, que implementa la estratégia de deducción forward chaining:

<img src="./imagenes/forward-chaining.png" width="550"/>

**Ejercicio 2:**

Implemente un código python con el algoritmo `pl_fc_entails`. Verifique cuáles de las letras 'p', 'q', 'r', o 's' se puede deducir a partir de la base de conocimiento creada anteriormente.

**Nota:** Observe que al inicializar `queue`como la lista de literales en los hechos, es importante que `queue` sea una copia. Si se usa la instrucción `queue=base.hechos`, entonces al modificar `queue` se modifica también `base.hechos`. Para que no suceda esto, se puede utilizar `queue=deepcopy(base.hechos)` (observe que hay que importar `deepcopy` de la libería `copy`).

In [7]:
from copy import deepcopy

def pl_fc_entails(KB, q):
    
    count = {r.nombre: len(r.antecedente) for r in KB.reglas}
    inferred = {r: False for r in KB.atomos}
    queue = deepcopy(KB.hechos)
    
    while len(queue) > 0:
        
        p = queue.pop()
        
        if p == q:
            return True
        
        if inferred[p] == False:
            inferred[p] = True
            
            for c in KB.reglas:
                
                if p in c.antecedente:
                    count[c.nombre] = count[c.nombre] - 1
                    
                if count[c.nombre] == 0:
                    queue.append(c.consecuente)
    
    return False
                       

In [8]:
print("El átomo p se deduce de la base?: {}".format(pl_fc_entails(base_ejemplo, 'p')))

print("El átomo q se deduce de la base?: {}".format(pl_fc_entails(base_ejemplo, 'q')))

print("El átomo r se deduce de la base?: {}".format(pl_fc_entails(base_ejemplo, 'r')))

print("El átomo s se deduce de la base?: {}".format(pl_fc_entails(base_ejemplo, 's')))

El átomo p se deduce de la base?: True
El átomo q se deduce de la base?: False
El átomo r se deduce de la base?: True
El átomo s se deduce de la base?: True


La respuesta debe ser:

```
El átomo p se deduce de la base?: True
El átomo q se deduce de la base?: False
El átomo r se deduce de la base?: True
El átomo s se deduce de la base?: True
```

---

## Backward chaining <a class="anchor" id="backward"></a>

([Volver al inicio](#inicio))

El siguiente es el pseudocódigo de la función `and_or_graph_search`, el cual implementa la estratégia de deducción backward chaining:

<img src="./imagenes/and-or-search.png" width="520"/>

**Ejercicio 3:**

Implemente un código python con el algoritmo `and_or_graph_search`. Verifique cuáles de las letras 'p', 'q', 'r', o 's' se puede deducir a partir de la base de conocimiento creada anteriormente.

In [24]:
def and_or_graph_search(objetivo, base):
    
    return or_search(objetivo, base, [])


def or_search(consecuente, base, camino):
    
    if base.test_objetivo(consecuente):
        return True
    
    elif consecuente in camino:
        return False
    
    for regla in base.reglas_aplicables(consecuente):
        
        camino2 = [consecuente] + camino
        resu = and_search(regla.antecedente, base, camino2)
        
        if resu == True:
            return True
    
    return False


def and_search(literales, base, camino):

    for consecuente in literales:
        
        plan = or_search(consecuente, base, camino)
        
        if plan == False:
            return False
    
    return True


In [25]:
print("El átomo p se deduce de la base?: {}".format(and_or_graph_search('p', base_ejemplo)))

print("El átomo q se deduce de la base?: {}".format(and_or_graph_search('q', base_ejemplo)))

print("El átomo r se deduce de la base?: {}".format(and_or_graph_search('r', base_ejemplo)))

print("El átomo s se deduce de la base?: {}".format(and_or_graph_search('s', base_ejemplo)))

El átomo p se deduce de la base?: True
El átomo q se deduce de la base?: False
El átomo r se deduce de la base?: True
El átomo s se deduce de la base?: True


La respuesta debe ser:

```
El átomo p se deduce de la base?: success
El átomo q se deduce de la base?: failure
El átomo r se deduce de la base?: success
El átomo s se deduce de la base?: success
```

---

## TELL y ASK <a class="anchor" id="tell"></a>

([Volver al inicio](#inicio))

Vamos a ver ahora cómo usar una base de conocimiento. Para ello, usaremos un ejemplo. Considere el *mundo de los bloques*, el cual consiste en un conjunto de bloques de forma cúbica, los cuales se encuentran encima de una mesa arbitrariamente grande. Los bloques se pueden apilar, pero solo se puede poner un bloque directamente encima de otro. El agente puede levantar un bloque y moverlo a otra posición, ya sea encima de la mesa o encima de otro bloque. El brazo no puede levantar un bloque que tiene otro encima.

<img src="./imagenes/bloques2.png" width="auto"/>

Podemos crear una base de conocimiento que describa el mundo en la figura anterior. Por ejemplo, sabemos que:

* $A$ está debajo de $C$.
* $B$ está a la izquierda de $A$.
* $C$ es morado.

Podemos usar la siguiente representación de estos hechos:

* `debajo(a,c)`
* `izquierda(b,a)`
* `morado(c)`

También sabemos, por sentido común, que:

* Si $B$ está a la izquierda de $A$, entonces $A$ está a la derecha de $B$.
* Si $C$ es morado, entonces no es verde.

Podemos usar la siguiente representación de estas reglas:

* `izquierda(b,a)>derecha(a,b)`
* `morado(c)>-verde(c)`

**La función TELL:**

La función TELL nos sirve para actualizar la base de conocimiento. Le podemos decir que una fórmula es verdadera. La función reconoce si la fórmula es un hecho o una regla, y lo incluye en el lugar apropiado (evitando repeticiones).

In [26]:
from logica import LPQuery

base = LPQuery([])
base.TELL('debajo(a,c)')
base.TELL('izquierda(b,a)')
base.TELL('morado(c)')
base.TELL('izquierda(b,a)>derecha(a,b)')
base.TELL('morado(c)>-verde(c)')
print(base)

Hechos:
debajo(a,c)
izquierda(b,a)
morado(c)

Reglas:
izquierda(b,a)>derecha(a,b)
morado(c)>-verde(c)



**La función ASK:**

Una base de conocimiento sirve para hacer consultas. Para hacer una consulta, debemos tener un objetivo, que es un hecho que queremos conocer si se deduce o no de la base.

Usaremos la siguiente función ASK:

<img src="./imagenes/ask.png" width="520"/>

**Ejercicio 4:**

Implemente la función ASK para preguntarle a la base de conocimiento si sabemos que $C$ es verde, si NO sabemos que $C$ es naranja y si sabemos que $A$. está a la derecha de $B$.

In [27]:
def ASK(objetivo, valor, base):
    
    if and_or_graph_search(objetivo, base) == valor: 
        return True
    
    else:
        return False

print('Sabemos que C es verde?:', ASK('verde(c)', 'success', base))
print('No sabemos que C es naranja?:', ASK('naranja(c)', 'failure', base))
print('Sabemos que A está a la derecha de B?:', ASK('derecha(a,b)', 'success', base))

Sabemos que C es verde?: False
No sabemos que C es naranja?: False
Sabemos que A está a la derecha de B?: False


**Nota:**

La respuesta debe ser:
```
Sabemos que C es verde?: False
No sabemos que C es naranja?: True
Sabemos que A está a la derecha de B?: True
```

---

Podemos generalizar la regla de los colores de la siguiente manera. Creamos un conjunto de bloques y otro de colores:

In [28]:
bloques = ['a', 'b', 'c']
colores = ['verde', 'morado', 'naranja']

Le decimos a la base de conocimiento que si un bloque es de un color, entonces no es del otro color:

In [29]:
for bloque in bloques:
    for color in colores:
        for otro_color in [c for c in colores if c != color]:
            base.TELL(f'{color}({bloque})>-{otro_color}({bloque})')

**Ejercicio 5:**

Dígale a la base de conocimiento el color de los bloques y consulte si sabemos que $A$ no es morado.

In [36]:
base.TELL('verde(b)')
base.TELL('morado(a)')

print("A no es morado:", ASK('morado(a)', 'success', base))

A no es morado False


**Ejercicio 6:**

En este ejercicio se pide hacer lo siguiente:

* Generalice las reglas de que si un bloque está debajo de otro, este último está arriba del primero. 
* Consulte si sabemos que $C$ está arriba de $A$.

In [ ]:
for bloque in bloques:
    
    for otro_bloque in ()

---

**Ejercicio 7:**

En este ejercicio se pide hacer lo siguiente:

* Observe que si un bloque está arriba o debajo de otro, entonces el primero hereda todas las relaciones de izquierda y derecha del segundo. Incluya estas reglas en la base de conocimiento. 
* Consulte si sabemos que $C$ está a la derecha de $B$.

---

## En este notebook usted aprendió

* Las reglas como un fragmento de la lógica proposicional para la deducción automática.
* El algoritmo de deducción mediante forward-chaining.
* El algoritmo de deducción mediante backward-chaining.
* El uso de reglas y de las bases de conocimiento para obtener nueva información.
